# Ethereum

Using viem

## Concepts


### networks

https://ethereum.org/developers/docs/networks/

* mainnet
* sepolia
* hoodi
* ephemery

In [ ]:
const chains = await import("viem/chains");
console.log(chains['mainnet'])
console.log(chains["sepolia"]);
console.log(chains["hoodi"]);
console.log(chains["ephemery"]);
